# Detecting pretraining bias with SageMaker Clarify

<img align="left" width="130" src="https://raw.githubusercontent.com/PacktPublishing/Amazon-SageMaker-Cookbook/master/Extra/cover-small-padded.png"/>

This notebook contains the code to help readers work through one of the recipes of the book [Machine Learning with Amazon SageMaker Cookbook: 80 proven recipes for data scientists and developers to perform ML experiments and deployments](https://www.amazon.com/Machine-Learning-Amazon-SageMaker-Cookbook/dp/1800567030)

### How to do it...

In [ ]:
%store -r s3_bucket_name
%store -r prefix
%store -r training_data_path

In [ ]:
import sagemaker

session = sagemaker.Session()
region = session.boto_region_name
role = sagemaker.get_execution_role()

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!mkdir -p tmp

In [ ]:
s3_training_data_path = training_data_path
s3_output_path = f"s3://{s3_bucket_name}/{prefix}/output"

In [ ]:
!aws s3 cp {s3_training_data_path} tmp/training_data.csv

In [ ]:
training_data = pd.read_csv("tmp/training_data.csv")
training_data

In [ ]:
from sagemaker import clarify

processor = clarify.SageMakerClarifyProcessor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    sagemaker_session=session
)

In [ ]:
data_config = clarify.DataConfig(
    s3_data_input_path=s3_training_data_path,
    s3_output_path=s3_output_path,
    label='approved',
    headers=training_data.columns.to_list(),
    dataset_type='text/csv'
)

In [ ]:
bias_config = clarify.BiasConfig(
    label_values_or_threshold=[1],
    facet_name='sex',
)

In [ ]:
%%time

processor.run_pre_training_bias(
    data_config=data_config, 
    data_bias_config=bias_config,
    methods=['CI']
)

In [ ]:
output = processor.latest_job.outputs[0]
output_destination = output.destination
output_destination

In [ ]:
!aws s3 cp {output_destination}/ tmp/ --recursive

In [ ]:
!ls -lahF tmp/

In [ ]:
!cat tmp/analysis.json